### Final Project: Fuzzy matching of movies and entertainment data

In [1]:
from google.cloud import bigquery

In [2]:
%load_ext google.cloud.bigquery

/Users/dylanlam/miniconda3/lib/python3.11/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


### Step 1: Environment Setup

We define the key configuration values for:
- Project ID
- Dataset name for final outputs (`fin_movies_entertainment`)
- BigQuery region (`us-central1`)
- Vertex AI connection ID
- Gemini & Embedding model names


In [3]:
project_id = "dylanericsp25"
dataset = "fin_movies_entertainment"
region = "us-central1"
connection_id = "vertex-connection" # BQ requires a connection to call the model in Vertex
embedding_model = "text-embedding-005"  # latest gecko embeddings model as of 04/18/25
gemini_model = "gemini-2.5-flash-preview-04-17" # latest gemini flash model as of 04/18/25

### Step 2: Create BigQuery Dataset

We create the final dataset in BigQuery (`fin_movies_entertainment`) where all embeddings and cluster outputs will be stored.


In [4]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset dylanericsp25.fin_movies_entertainment


### Step 3: Create Vertex AI Connection & Grant Permissions

BigQuery must be authorized to call Vertex AI to generate embeddings. We:
1. Create a `CLOUD_RESOURCE` connection in BigQuery.
2. Grant `Vertex AI User` permissions to the service account used by BigQuery for model access.


In [5]:
!bq mk --connection --location=$region --project_id=$project_id \
    --connection_type=CLOUD_RESOURCE $connection_id



Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update

BigQuery error in mk operation: Already Exists: Connection
projects/960753582873/locations/us-central1/connections/vertex-connection


In [6]:
!bq show --connection 798706649102.us-central1.vertex-connection

Connection 798706649102.us-central1.vertex-connection

                     name                      friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 -------------------------------------------- -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  798706649102.us-central1.vertex-connection                                13 Apr 17:14:41   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-798706649102-xf2e@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



In [7]:
!gcloud projects add-iam-policy-binding $project_id --member='serviceAccount:bqcx-798706649102-xf2e@gcp-sa-bigquery-condel.iam.gserviceaccount.com' \
  --role='roles/aiplatform.user' --no-user-output-enabled

### Step 4: Create Remote Embedding Model in BigQuery

This creates a BigQuery model that points to the Vertex AI embedding model `text-embedding-005`. We’ll use this model to embed movie titles into high-dimensional vectors.


In [18]:
%%bigquery
create or replace model fin_movies_entertainment.embedding_model
  remote with connection `projects/dylanericsp25/locations/us-central1/connections/vertex-connection`
  options (endpoint = 'text-embedding-005');

Query is running:   0%|          |

""


### Step 5: Generate Embeddings for Netflix Titles

We use `ML.GENERATE_EMBEDDING` to generate vector representations of movie titles from our staging table `movies_entertainment_stg.netflix_movies_and_tvshows`. The resulting vectors are stored in a temporary table.


In [9]:
%%bigquery --location=us-central1
CREATE OR REPLACE TABLE fin_movies_entertainment.temp_embeddings AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `fin_movies_entertainment.embedding_model`,
  (
    SELECT 
      show_id, 
      title,
      title AS content
    FROM `dylanericsp25.movies_entertainment_stg.netflix_movies_and_tvshows`
    WHERE title IS NOT NULL
  )
);



Query is running:   0%|          |

""


In [10]:
%%bigquery --location=us-central1
SELECT *
FROM fin_movies_entertainment.temp_embeddings
LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,show_id,title,content
0,"[-0.038169655948877335, -0.019176844507455826,...","{""token_count"":1,""truncated"":false}",,s1945,Mann,Mann
1,"[-0.014502783305943012, -0.05777623504400253, ...","{""token_count"":1,""truncated"":false}",,s1065,Signal,Signal
2,"[-0.015388846397399902, -0.008072860538959503,...","{""token_count"":1,""truncated"":false}",,s2822,Kingdom,Kingdom
3,"[-0.03709067404270172, 0.020973458886146545, -...","{""token_count"":1,""truncated"":false}",,s8090,Stephanie,Stephanie
4,"[-0.06244494765996933, -0.060874972492456436, ...","{""token_count"":1,""truncated"":false}",,s2717,Brothers,Brothers
5,"[-0.05633598193526268, -0.007020583376288414, ...","{""token_count"":1,""truncated"":false}",,s3798,Joy,Joy
6,"[-0.03794030472636223, -0.018821218982338905, ...","{""token_count"":1,""truncated"":false}",,s6549,Dad,Dad
7,"[-0.06568083167076111, -0.00973602756857872, 0...","{""token_count"":1,""truncated"":false}",,s2755,Greater,Greater
8,"[-0.056740060448646545, -0.03770735114812851, ...","{""token_count"":1,""truncated"":false}",,s5389,46,46
9,"[-0.04382584989070892, 0.0012018682900816202, ...","{""token_count"":1,""truncated"":false}",,s7140,Jonathan,Jonathan


### Step 6: Final Embeddings Table

We copy only the relevant columns (title, show_id, embedding) from the temp table into a final clean table. This table will be used for distance calculations.


In [11]:
%%bigquery --location=us-central1
CREATE OR REPLACE TABLE fin_movies_entertainment.embeddings AS
SELECT
  show_id,
  title,
  ml_generate_embedding_result AS embedding
FROM fin_movies_entertainment.temp_embeddings;


Query is running:   0%|          |

""


In [12]:
%%bigquery --location=us-central1
CREATE OR REPLACE TABLE fin_movies_entertainment.embeddings_flattened AS
SELECT
  show_id,
  title,
  dim_value,
  dim_index
FROM fin_movies_entertainment.embeddings,
UNNEST(embedding) AS dim_value WITH OFFSET AS dim_index;


Query is running:   0%|          |

""


### Step 7: Preview Embedding Output

We preview the first few rows of the generated embeddings to ensure they were created successfully.


In [19]:
%%bigquery --location=us-central1
CREATE OR REPLACE TABLE fin_movies_entertainment.embedding_distances AS
WITH base AS (
  SELECT * 
  FROM fin_movies_entertainment.embeddings
  LIMIT 300  
),
pairs AS (
  SELECT
    a.show_id AS show_id_1,
    a.title AS title_1,
    b.show_id AS show_id_2,
    b.title AS title_2,
    SQRT(SUM(POW(a.embedding[i] - b.embedding[i], 2))) AS euclidean_distance
  FROM base a
  JOIN base b
  ON a.show_id != b.show_id
  CROSS JOIN UNNEST(GENERATE_ARRAY(0, ARRAY_LENGTH(a.embedding)-1)) AS i
  GROUP BY show_id_1, title_1, show_id_2, title_2
)
SELECT * FROM pairs
WHERE euclidean_distance < 1.0 
ORDER BY euclidean_distance ASC;



Query is running:   0%|          |

""


In [20]:
%%bigquery --location=us-central1

CREATE OR REPLACE TABLE fin_movies_entertainment.embedding_sample AS
SELECT *
FROM fin_movies_entertainment.embeddings
LIMIT 300;



Query is running:   0%|          |

""


### Step 8: Compute Pairwise Embedding Distances

We compute the **Euclidean distance** between each pair of titles. Only pairs with `distance < 1.0` are retained as “similar enough” — these will be our fuzzy matches.

Note: We limit the dataset here to avoid BigQuery CPU quota errors.


In [15]:
%%bigquery --location=us-central1

CREATE OR REPLACE TABLE fin_movies_entertainment.embedding_distances AS
SELECT
  A.show_id AS show_id_1,
  A.title AS title_1,
  B.show_id AS show_id_2,
  B.title AS title_2,
  SQRT(SUM(POW(A.embedding[i] - B.embedding[i], 2))) AS euclidean_distance
FROM
  fin_movies_entertainment.embedding_sample A,
  fin_movies_entertainment.embedding_sample B,
  UNNEST(GENERATE_ARRAY(0, ARRAY_LENGTH(A.embedding) - 1)) AS i
WHERE
  A.show_id < B.show_id
GROUP BY
  show_id_1, title_1, show_id_2, title_2
HAVING
  euclidean_distance < 1.0;


Query is running:   0%|          |

""


### Step 9: View Fuzzy Matched Titles

This final table shows pairs of movie titles that were detected as close matches based on vector similarity. These results can be used to deduplicate, standardize, or enrich metadata downstream.


In [21]:
%%bigquery --location=us-central1

SELECT *
FROM fin_movies_entertainment.embedding_distances
ORDER BY euclidean_distance ASC
LIMIT 50;


Query is running:   0%|          |

Downloading:   0%|          |

,show_id_1,title_1,show_id_2,title_2,euclidean_distance
0,s5965,9-Feb,s3997,9-Feb,0.000000
1,s4523,22-Jul,s5966,22-Jul,0.000000
2,s5966,22-Jul,s4523,22-Jul,0.000000
3,s3963,15-Aug,s5967,15-Aug,0.000000
4,s5967,15-Aug,s3963,15-Aug,0.000000
5,s3997,9-Feb,s5965,9-Feb,0.000000
6,s4069,A Second Chance,s5390,A 2nd Chance,0.225115
7,s5390,A 2nd Chance,s4069,A Second Chance,0.225115
8,s852,99 Songs (Tamil),s853,99 Songs (Telugu),0.380787
9,s853,99 Songs (Telugu),s852,99 Songs (Tamil),0.380787


In [17]:
%%bigquery --location=us-central1

CREATE OR REPLACE TABLE fin_movies_entertainment.deduplicated_titles AS
SELECT 
  LEAST(title_1, title_2) AS canonical_title,
  GREATEST(title_1, title_2) AS duplicate_title,
  euclidean_distance
FROM fin_movies_entertainment.embedding_distances
WHERE euclidean_distance < 0.5  


Query is running:   0%|          |

""


## Step 10: Conclusion and Reflection


---

### Conclusion: Embedding-Based Fuzzy Matching Summary

In this project, I explored fuzzy matching using embedding vectors with BigQuery + Vertex AI.

- **Input Dataset**: Titles from the Netflix Movies & TV Shows staging table (`stg_netflix_movies_and_tvshows`).
- **Embedding Generation**: Used the `text-embedding-005` model through BigQuery ML and Vertex AI.
- **Distance Calculation**: Used Euclidean distance to measure semantic similarity between movie titles.
- **Deduplication Logic**: Identified pairs of titles with distances < 0.5 and mapped them to a canonical title.

#### Key Insights:
- Found logical groupings such as:
  - `"99 Songs (Tamil)"` and `"99 Songs (Telugu)"`
  - `"13 Reasons Why"` and `"13 Reasons Why: Beyond the Reasons"`
  - Calendar titles like `"22-Jul"` and `"15-Aug"`
- Embedding-based matching provided **semantic matches**, outperforming string-only approaches (like `GROUP BY` or `LIKE` filters).

#### Would I use this in production?
Yes — embeddings scale better and avoid hallucinations. I'd integrate this into an intermediate model layer for deduplication and record linkage, especially when titles are short, noisy, or inconsistently labeled.

